In [141]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import classification_report, mean_squared_error
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping

In [142]:
np.random.seed(42)

In [143]:
df = pd.read_csv("../data/cbsa_data_realpageflag.csv")

C:\Users\ahmad\AppData\Local\Temp\ipykernel_40832\636769717.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/cbsa_data_realpageflag.csv")


In [144]:
df.columns

Index(['propertyId', 'propertyName', 'zipCode', 'address', 'city', 'state',
       'averageSquareFootage', 'stories', 'stable', 'class', 'latitude',
       'longitude', 'imageHero', 'heroSource', 'submarketName', 'unitcount',
       'yearBuilt', 'siteStatus', 'type', 'MPF-ANN-RENT-CHG',
       'MPF-HIST-ASK-RENT', 'MPF-HIST-AVG-ASK-RPSF', 'MPF-HIST-CONC-RATIO',
       'MPF-OCC', 'MPF-RENT', 'MPF-RPSF', 'data_index', 'cbsa_code', 'status',
       'SH-HIST-ASK-RENT-PB', 'SH-HIST-RENT-PB', 'SH-HIST-CONC-RATIO-PB',
       'SH-HIST-OCCUPANCY', 'SH-HIST-RPSF-PB', 'SH-HIST-AVG-ASK-RPSF-PB',
       'SH-HIST-ANN-RENT-PB-CHG', 'uses_realpage'],
      dtype='object')

In [145]:
# developing yearsold column (this is just the year the property was built minus the current year)
current_year = datetime.now().year
df['yearsold'] = current_year - df['yearBuilt']

In [146]:
# selecting relevant columns to use in the model
cols = [
    'cbsa_code', 'averageSquareFootage', 'stories', 'class', 'unitcount', 'yearsold', 'MPF-OCC', 'MPF-RPSF', 'uses_realpage'
]
df.dropna(subset=cols,inplace=True)

In [147]:
def print_null_counts(df, columns):
    for col in columns:
        null_count = df[col].isnull().sum()
        print(f"{col}: {null_count} null(s)")


columns_to_check = ['averageSquareFootage', 'stories', 'cbsa_code', 'class', 'yearsold', 'MPF-OCC', 'unitcount','MPF-HIST-CONC-RATIO','uses_realpage','MPF-RPSF']
print_null_counts(df, columns_to_check)

#while the concentration ratio would be phenomenal to use, excluding 25167 rows would be very detrimental to our dataset. this is over 75% of our dataset and would essentially remove most of our cbsa's include ones we have realpage users for

averageSquareFootage: 0 null(s)
stories: 0 null(s)
cbsa_code: 0 null(s)
class: 0 null(s)
yearsold: 0 null(s)
MPF-OCC: 0 null(s)
unitcount: 0 null(s)
MPF-HIST-CONC-RATIO: 27308 null(s)
uses_realpage: 0 null(s)
MPF-RPSF: 0 null(s)


# Feature Engineering & Exploratory Data Analysis (EDA)

Before developing our models, we want to validate parts of our dataset and clean parts that dont make sense. These include the following:

- **Excluding Student Housing:**  
   - **What:** Filter out any properties categorized as student housing.
   - **Why:** Student housing often follows different market dynamics that do not align with our target analysis of RealPage usage and property rent. Removing these properties helps focus our model on the residential segments of interest.

-  **Excluding CBSAs with No RealPage Usage:**  
   - **What:** Remove any Core Based Statistical Areas (CBSAs) that do not have at least one property using RealPage.
   - **Why:** This ensures that our analysis only includes markets where RealPage is active. By doing so, we improve the relevance and reliability of our insights and predictions. We also want to know how the dynamics of a market will change with the treatment of it being a RealPage user, so a market without any RealPage usage would not really be beneficial for our cause.

- **Integrating External Market Share Data:**  
  We incorporate an external dataset from a research paper/article that provides every CBSA code along with RealPage's estimated market share. This dataset will serve as a benchmark to compare against our own estimates.

- **Creating a Proxy for Market Share per CBSA:**  
  Using our internal dataset, we compute a proxy for market share at the CBSA level. This proxy is calculated based on the proportion of RealPage users (flagged properties) relative to the total number of properties within each CBSA. It offers a localized estimation of RealPage's market penetration.

- **Comparing Our Proxy with the External Benchmark:**  
  We assess the similarity between our computed market share proxy and the research's market share data. This comparison will help validate the accuracy of our proxy, ensuring our estimations are consistent with external insights.

- **Filtering the Dataset for Reliability:**  
  - **Exclude CBSAs with No RealPage Users:**  
    To focus on relevant markets, any CBSA with zero flagged RealPage users will be excluded.
  - **Exclude CBSAs with Fewer than 2 Properties:**  
    CBSAs with less than 2 properties are removed to avoid noise and ensure statistical significance in our analysis.

In [148]:
# Excluding student housing
df = df[~df.get('type', '').astype(str).str.contains("Student", na=False)]

In [149]:
print(f" Number of unique CBSAs: {len(df['cbsa_code'].unique())}")

 Number of unique CBSAs: 298


In [150]:
# Excluding cbsas that do not have at least 1 realpage user and less than 2 properties
realpage_usage_by_cbsa = df.groupby('cbsa_code').agg(
    total_properties=('cbsa_code', 'count'),
    realpage_count=('uses_realpage', 'sum')
).reset_index()

valid_cbsa = realpage_usage_by_cbsa[(realpage_usage_by_cbsa['realpage_count'] >= 1) & (realpage_usage_by_cbsa['total_properties'] >= 2)]

df = df[df['cbsa_code'].isin(valid_cbsa['cbsa_code'])]

In [151]:
print(f" Number of unique CBSAs after the filter: {len(df['cbsa_code'].unique())}")

 Number of unique CBSAs after the filter: 177


In [152]:
#adding in dataset with researched market shares per cbsa
cbsa_rp_ms = pd.read_csv('msa_market_share_cbsa_matched_final.csv')

In [153]:
cbsa_rp_ms.head()

,MSA,Realpage Market Share,CBSA Code
0,"Austin-Round Rock-San Marcos, TX",46.004304,12420
1,"Raleigh-Cary, NC",42.650406,39580
2,"Gainesville, GA",34.424809,23580
3,"Durham-Chapel Hill, NC",33.856628,20500
4,"Dallas-Fort Worth-Arlington, TX",33.721714,19100


In [154]:
df_with_ms = df.merge(cbsa_rp_ms, left_on='cbsa_code', right_on='CBSA Code', how='left')

In [155]:
print(f"CBSA's without matched market shares: {len(df_with_ms[df_with_ms['Realpage Market Share'].isnull()]['cbsa_code'].unique())}")

CBSA's without matched market shares: 28


In [156]:
df_with_ms[df_with_ms['Realpage Market Share'].isnull()]['cbsa_code'].unique()

array([13900, 14020, 16300, 19340, 20420, 21780, 23940, 24780, 25260,
       25620, 29020, 29300, 29540, 30700, 31740, 33540, 33740, 35100,
       35980, 37900, 39420, 40420, 40580, 42940, 44180, 48060, 48780,
       49660], dtype=int64)

In [157]:
market_share_proxy = df.groupby('cbsa_code',as_index=False).agg(props=('propertyId','count')
                                           ,realpage_users=('uses_realpage','sum'))
market_share_proxy['market_share'] = market_share_proxy['realpage_users']/market_share_proxy['props']

In [158]:
error_rate_df = market_share_proxy.merge(cbsa_rp_ms,left_on='cbsa_code',right_on='CBSA Code',how='left')
error_rate_df['market_share_proxy'] = error_rate_df['market_share'] * 100

In [159]:
mse_series = (error_rate_df[error_rate_df['CBSA Code'].isnull() == False]['Realpage Market Share'] - error_rate_df[error_rate_df['CBSA Code'].isnull() == False]['market_share_proxy'])**2

In [160]:
print(f"RSME of proxy vs research market shares: {(mse_series.sum()/len(mse_series))**(1/2)}")
print(f"Standard Deviation of reseached market shares:{cbsa_rp_ms['Realpage Market Share'].std()}")

RSME of proxy vs research market shares: 10.898890574316335
Standard Deviation of reseached market shares:8.571298144362245


# Model 1: Predicting the Usage of RealPage

In this section, we'll develop a binary classification model to predict whether a property uses RealPage. The key steps include:

- **Preprocessing & Feature Engineering:**
  - Compute `yearsold` from the `yearBuilt` column (Current year - yearBuilt).  
  - Standardize numerical features using StandardScaler.
  - One-hot encode categorical features (CBSA code and class).
  - Eliminate any NA rows

- **Modeling:**  
  - Use logistic regression to estimate the propensity (probability) that a property uses RealPage.
  - Perform an 80/20 train-test split for model evaluation.

- **Evaluation:**  
  - Generate a classification report with metrics like precision, recall, and F1-score.

Goal: Prompt a user for input variables and make a prediction based on the variables whether it is a realpage user or not


Columns used in model:
<br>  - `cbsa_code`
<br>    - `averageSquareFootage`
<br>    - `stories`
<br>    - `class`
<br>    - `unitcount`
<br>    - `yearsold`
<br>    - `MPF-OCC`
<br>    - `MPF-RPSF`
<br>    - `uses_realpage`

Please refer to the website (https://uc-berkeley-i-school.github.io/realpage-collusion/) for definitions of the columns.


In [161]:
#creating our treatment and control groups
features_treatment = ['averageSquareFootage', 'stories', 'cbsa_code', 'class', 'yearsold', 'MPF-OCC', 'unitcount', 'MPF-RPSF']
X_treatment = df[features_treatment]
y_treatment = df['uses_realpage']

# assigning numeric and categorical features
numeric_features_treatment = ['averageSquareFootage', 'stories', 'yearsold', 'MPF-OCC', 'unitcount', 'MPF-RPSF']
categorical_features_treatment = ['cbsa_code', 'class']

In [162]:
preprocessor_treatment = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features_treatment),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features_treatment)
    ]
)

In [163]:
clf_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor_treatment),
    ('classifier', LogisticRegression(solver='liblinear', random_state=42))
])

In [164]:
X_treat_train, X_treat_test, y_treat_train, y_treat_test = train_test_split(
    X_treatment, y_treatment, test_size=0.2, random_state=42
)

In [165]:
clf_pipeline.fit(X_treat_train, y_treat_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['averageSquareFootage',
                                                   'stories', 'yearsold',
                                                   'MPF-OCC', 'unitcount',
                                                   'MPF-RPSF']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['cbsa_code', 'class'])])),
                ('classifier',
                 LogisticRegression(random_state=42, solver='liblinear'))])

In [166]:
y_treat_pred = clf_pipeline.predict(X_treat_test)

In [180]:
print("Treatment Model Classification Report:")
print(classification_report(y_treat_test, y_treat_pred))

Treatment Model Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.99      0.93      5771
           1       0.41      0.03      0.05       877

    accuracy                           0.87      6648
   macro avg       0.64      0.51      0.49      6648
weighted avg       0.81      0.87      0.81      6648



# Model 2: Predicting the Price of Properties

In this section, our goal is to build a regression model to predict the rent per square foot (MPF-RPSF) of properties. The process involves:

- **Preprocessing & Feature Engineering:**
  - Compute `yearsold` from the `yearBuilt` column (Current year - yearBuilt).  
  - Standardize numerical features using StandardScaler.
  - One-hot encode categorical features (CBSA code and class).
  - Eliminate any NA rows

- **Modeling:**  
  - Implement a Random Forest Regressor for robust prediction.
  - Split the data into an 80/20 train-test set.

- **Evaluation:**  
  - Assess the model using the Mean Squared Error (MSE) metric.

This setup will allow us to test, tweak, and improve both models based on the performance and insights we gain from the evaluation.

Goal: Allow a user to prompt the model and see how the models prediction of price changes as they change the realpage usage variable, while keeping other variables constant.

Columns used in model:
<br>    - `cbsa_code`
<br>    - `averageSquareFootage`
<br>    - `stories`
<br>    - `class`
<br>    - `unitcount`
<br>    - `yearsold`
<br>    - `MPF-OCC`
<br>    - `MPF-RPSF`
<br>    - `uses_realpage`

Please refer to the website (https://uc-berkeley-i-school.github.io/realpage-collusion/) for definitions of the columns.

In [168]:
#defining features and target
features_rent = ['averageSquareFootage', 'stories', 'cbsa_code', 'class', 'yearsold', 'MPF-OCC', 'unitcount','uses_realpage']
X_rent = df[features_rent]
y_rent = df['MPF-RPSF']

#assigning numeric and categorical features
numeric_features_rent = ['averageSquareFootage', 'stories', 'yearsold', 'MPF-OCC', 'unitcount','uses_realpage']
categorical_features_rent = ['cbsa_code', 'class']

In [169]:
#develop a preprocessor for for the rent model
preprocessor_rent = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features_rent),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features_rent)
    ]
)

In [170]:
#build pipeline for the regression model, we will use random forest regressor. will run through preprossor first, then through the regressor model.
reg_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor_rent),
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))
])

In [171]:
X_rent_train, X_rent_test, y_rent_train, y_rent_test = train_test_split(
    X_rent, y_rent, test_size=0.2, random_state=42
)

In [202]:
reg_pipeline.fit(X_rent_train, y_rent_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['averageSquareFootage',
                                                   'stories', 'yearsold',
                                                   'MPF-OCC', 'unitcount',
                                                   'uses_realpage']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['cbsa_code', 'class'])])),
                ('regressor', RandomForestRegressor(random_state=42))])

In [173]:
y_rent_pred = reg_pipeline.predict(X_rent_test)
mse = mean_squared_error(y_rent_test, y_rent_pred)
print("Rent Model Mean Squared Error:", mse)

Rent Model Mean Squared Error: 0.04848786969718381


In [179]:
#finding feature importance for the random forest regressor model
preprocessor_reg_1 = reg_pipeline.named_steps['preprocessor']
regressor_reg_1 = reg_pipeline.named_steps['regressor']

num_features_reg_1 = numeric_features_rent

cat_transformer_reg_1 = preprocessor_reg_1.named_transformers_['cat']
cat_features_reg_1 = list(cat_transformer_reg_1.get_feature_names_out(categorical_features_rent))

all_feature_names_reg_1 = num_features_reg_1 + cat_features_reg_1

importances_reg_1 = regressor_reg_1.feature_importances_

feature_importance_reg1_df = pd.DataFrame({
    'feature': all_feature_names_reg_1,
    'importance': importances_reg_1
}).sort_values(by='importance', ascending=False)

cbsa_mask = feature_importance_reg1_df['feature'].str.startswith('cbsa_code')
cbsa_agg = feature_importance_reg1_df.loc[cbsa_mask, 'importance'].sum()

feature_importance_reg1_df = feature_importance_reg1_df.loc[~cbsa_mask]

agg__cbsa_row = pd.DataFrame({'feature': ['cbsa_code (aggregated)'], 'importance': [cbsa_agg]})
feature_importance_reg1_df = pd.concat([feature_importance_reg1_df, agg__cbsa_row], ignore_index=True)

class_mask = feature_importance_reg1_df['feature'].str.startswith('class')
class_agg = feature_importance_reg1_df.loc[class_mask, 'importance'].sum()

feature_importance_reg1_df = feature_importance_reg1_df.loc[~class_mask]

agg_class_row = pd.DataFrame({'feature': ['class (aggregated)'], 'importance': [class_agg]})
feature_importance_reg1_df = pd.concat([feature_importance_reg1_df, agg_class_row], ignore_index=True)

print("Feature Importances for model 1:")
print(feature_importance_reg1_df.sort_values('importance',ascending=False))

Feature Importances for model 1:
                  feature  importance
6  cbsa_code (aggregated)    0.543746
0                 stories    0.215968
7      class (aggregated)    0.121429
1    averageSquareFootage    0.069682
2                yearsold    0.017268
3               unitcount    0.017060
4                 MPF-OCC    0.013397
5           uses_realpage    0.001450


# Try it Yourself! Predict the RPSF

In this section, we're building an interactive function that allows a user to input key property details. Based on these inputs, our regression model will predict the rent per square foot (RPSF). This serves as a prototype for the similar functionality we plan to integrate into our website.

## What Does the Function Do?

- **User Prompts:**  
  The function asks the user to enter several input variables, including:
  - Average square footage
  - Number of stories
  - CBSA code (categorical)
  - Property class (categorical)
  - Year built (from which we calculate the property's age)
  - MPF-OCC (occupancy)
  - Unit count

- **Output:**  
  The function displays the inputs the user provided and then outputs the prediction in a clear format, e.g.,  
  `Predicted RPSF: $XXXX`

## How This Relates to Our Website

This interactive function is a small-scale prototype of what we plan to implement on our website. On the live site:
- Users can navigate to the Try it Yourself section within the website and test inputs
- The website backend will process these inputs into our model using sagemaker
- The predicted rent per square foot will be displayed to the user immediately.


In [20]:
def predict_rpsf(model):
    """
    Prompts the user for input variables, displays the inputs,
    and outputs the predicted rent per square foot.
    """
    try:
        averageSquareFootage = float(input("Enter average square footage: "))
        stories = float(input("Enter number of stories: "))
        cbsa_code = input("Enter CBSA code: ")
        property_class = input("Enter property class: ")
        yearbuilt = int(input("Enter the year built: "))
        MPF_OCC = float(input("Enter MPF-OCC (occupancy): "))
        unitcount = float(input("Enter unit count: "))
        uses_realpage = input("Enter uses_realpage: ")
    except Exception as e:
        print(f"Error in input: {e}")
        return

    current_year = datetime.now().year
    yearsold = current_year - yearbuilt

    print("\nUser Inputs:")
    print(f"  Average Square Footage: {averageSquareFootage}")
    print(f"  Stories: {stories}")
    print(f"  CBSA Code: {cbsa_code}")
    print(f"  Property Class: {property_class}")
    print(f"  Year Built: {yearbuilt}")
    print(f"  Years Old: {yearsold}")
    print(f"  MPF-OCC (Occupancy): {MPF_OCC}")
    print(f"  Unit Count: {unitcount}")
    print(f"  Uses RealPage: {uses_realpage}\n")
    
    input_df = pd.DataFrame({
        'averageSquareFootage': [averageSquareFootage],
        'stories': [stories],
        'cbsa_code': [cbsa_code],
        'class': [property_class],
        'yearsold': [yearsold],
        'MPF-OCC': [MPF_OCC],
        'unitcount': [unitcount],
        'uses_realpage': [uses_realpage]
    })
    
    prediction = model.predict(input_df)
    predicted_rpsf = prediction[0]
    
    print(f"Predicted RPSF: ${predicted_rpsf:.2f}")
    return predicted_rpsf


In [ ]:
predict_rpsf(reg_pipeline)


User Inputs:
  Average Square Footage: 890.0
  Stories: 2.0
  CBSA Code: 10180
  Property Class: A
  Year Built: 1994
  Years Old: 31
  MPF-OCC (Occupancy): 0.964
  Unit Count: 190.0
  Uses RealPage: 1

Predicted RPSF: $1.58


1.5804899999999988

In [181]:
"""
Next steps for me to work on:
-   try different models
-   do some hyperparameter tuning
-   put in sagemaker and try to hit the model via API
"""

'\nNext steps for me to work on:\n-   try different models\n-   do some hyperparameter tuning\n-   put in sagemaker and try to hit the model via API\n'

## Model 3

Earlier, we found a proxy in our dataset for realpage's market share using the flagged users per cbsa in our dataset. This is our model will change:

- **Feature Engineering:**
  - Adding in a feature for market share using the proxy we developed
  - The proxy is found by the following calculation per CBSA: `Total Number of Flagged Realpage Users / Total Number of Property Listings`



In [194]:
market_share_proxy = market_share_proxy.rename(columns={'cbsa_code':'CBSA'})

In [195]:
model3_df = df.merge(market_share_proxy[['CBSA','market_share']],left_on='cbsa_code',right_on='CBSA',how='left')

In [196]:
#defining features and target
features_rent_reg_2 = ['averageSquareFootage', 'stories', 'cbsa_code', 'class', 'yearsold', 'MPF-OCC', 'unitcount','uses_realpage','market_share']
X_rent_reg_2 = model3_df[features_rent_reg_2]
y_rent_reg_2 = model3_df['MPF-RPSF']

#assigning numeric and categorical features
numeric_features_rent_reg_2 = ['averageSquareFootage', 'stories', 'yearsold', 'MPF-OCC', 'unitcount','uses_realpage','market_share']
categorical_features_rent_reg_2 = ['cbsa_code', 'class']

In [197]:
#develop a preprocessor for for the rent model
preprocessor_rent_reg_2 = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features_rent_reg_2),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features_rent_reg_2)
    ]
)

In [198]:
reg_pipeline_reg_2 = Pipeline(steps=[
    ('preprocessor', preprocessor_rent_reg_2),
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))
])

In [199]:
X_rent_train_reg_2, X_rent_test_reg_2, y_rent_train_reg_2, y_rent_test_reg_2 = train_test_split(
    X_rent_reg_2, y_rent_reg_2, test_size=0.2, random_state=42
)

In [201]:
reg_pipeline_reg_2.fit(X_rent_train_reg_2, y_rent_train_reg_2)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['averageSquareFootage',
                                                   'stories', 'yearsold',
                                                   'MPF-OCC', 'unitcount',
                                                   'uses_realpage',
                                                   'market_share']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['cbsa_code', 'class'])])),
                ('regressor', RandomForestRegressor(random_state=42))])

In [204]:
y_rent_pred_reg_2 = reg_pipeline_reg_2.predict(X_rent_test_reg_2)
mse_reg_2 = mean_squared_error(y_rent_test_reg_2, y_rent_pred_reg_2)
print("Rent Model Mean Squared Error:", mse_reg_2)

Rent Model Mean Squared Error: 0.040398723995711296


In [206]:
#finding feature importance for the random forest regressor model
preprocessor_reg_2 = reg_pipeline_reg_2.named_steps['preprocessor']
regressor_reg_2 = reg_pipeline_reg_2.named_steps['regressor']

num_features_reg_2 = numeric_features_rent_reg_2

cat_transformer_reg_2 = preprocessor_reg_2.named_transformers_['cat']
cat_features_reg_2 = list(cat_transformer_reg_2.get_feature_names_out(categorical_features_rent_reg_2))

all_feature_names_reg_2 = num_features_reg_2 + cat_features_reg_2

importances_reg_2 = regressor_reg_2.feature_importances_

feature_importance_reg2_df = pd.DataFrame({
    'feature': all_feature_names_reg_2,
    'importance': importances_reg_2
}).sort_values(by='importance', ascending=False)

cbsa_mask = feature_importance_reg2_df['feature'].str.startswith('cbsa_code')
cbsa_agg = feature_importance_reg2_df.loc[cbsa_mask, 'importance'].sum()

feature_importance_reg2_df = feature_importance_reg2_df.loc[~cbsa_mask]

agg__cbsa_row = pd.DataFrame({'feature': ['cbsa_code (aggregated)'], 'importance': [cbsa_agg]})
feature_importance_reg2_df = pd.concat([feature_importance_reg2_df, agg__cbsa_row], ignore_index=True)

class_mask = feature_importance_reg2_df['feature'].str.startswith('class')
class_agg = feature_importance_reg2_df.loc[class_mask, 'importance'].sum()

feature_importance_reg2_df = feature_importance_reg2_df.loc[~class_mask]

agg_class_row = pd.DataFrame({'feature': ['class (aggregated)'], 'importance': [class_agg]})
feature_importance_reg2_df = pd.concat([feature_importance_reg2_df, agg_class_row], ignore_index=True)

print("Feature Importances for model 2:")
print(feature_importance_reg2_df.sort_values('importance',ascending=False))

Feature Importances for model 2:
                  feature  importance
7  cbsa_code (aggregated)    0.365948
0            market_share    0.252231
8      class (aggregated)    0.193452
1                 stories    0.079133
2    averageSquareFootage    0.064340
3               unitcount    0.015716
4                yearsold    0.015670
5                 MPF-OCC    0.012320
6           uses_realpage    0.001190
